In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import layers


#depression tweets
dt = pd.read_csv("data/depression_tweets.tsv", sep="\t", index_col="tweet_id", parse_dates=[2])
#dt["created_at"] = pd.to_datetime(dt["created_at"])
n_depression = len(dt['user_id'].unique())
print(n_depression)

#sample tweets
st = pd.read_csv("data/sample_tweets.tsv", sep="\t", index_col="tweet_id", parse_dates=[2])
#st["created_at"] = pd.to_datetime(st["created_at"])
#filtered users
fu = pd.read_csv('filtered_user_lists.csv')
fu_tbt = pd.read_csv('filtered_user_lists_only_tpd_tbt.csv')
fu_maxtime = pd.read_csv('filtered_user_lists_only_tpd_maxtime.csv')
#tpd pairs
tpd_pairs = pd.read_csv('pairs_tpd_users.csv')
filtered_users = pd.concat([fu, fu_tbt, fu_maxtime], ignore_index=True)

KeyboardInterrupt: 

# final lstm tuning

In [1]:
import pandas as pd
from sktime.classification.hybrid import HIVECOTEV2
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support
from sklearn.neighbors import KNeighborsClassifier
import time

#depression tweets
dt = pd.read_csv("data/depression_tweets.tsv", sep="\t", index_col="tweet_id", parse_dates=[2])
#dt["created_at"] = pd.to_datetime(dt["created_at"])
n_depression = len(dt['user_id'].unique())
print(n_depression)

#sample tweets
st = pd.read_csv("data/sample_tweets.tsv", sep="\t", index_col="tweet_id", parse_dates=[2])
#st["created_at"] = pd.to_datetime(st["created_at"])

#filtered users
fu = pd.read_csv('filtered_user_lists.csv')
fu_tbt = pd.read_csv('filtered_user_lists_only_tpd_tbt.csv')
fu_maxtime = pd.read_csv('filtered_user_lists_only_tpd_maxtime.csv')
#tpd pairs
tpd_pairs = pd.read_csv('pairs_tpd_users.csv')
filtered_users = pd.concat([fu, fu_tbt, fu_maxtime], ignore_index=True)

603


In [2]:
j = 948
current = filtered_users.iloc[j]
print(current)
current_user_list = eval(current.user_list)

#get distribution of current filtered list
avg_dist = {}
current_tpds = tpd_pairs[tpd_pairs['user id'].isin(set(current_user_list))]
for avg in current_tpds['tweets per day'].unique():
    tmp_dist = float((current_tpds[current_tpds['tweets per day']==avg].count()/len(current_tpds)).iloc[0])
    avg_dist.update({str(avg): str(tmp_dist)})

dist_df = pd.DataFrame(avg_dist.items(), columns=['tweets per day', 'percentage'])
dist_df = dist_df.astype({'tweets per day': 'int32'})
dist_df = dist_df.astype({'percentage': 'float'})
dist_df = dist_df.sort_values(by=['tweets per day'])

#downsample current user list based on distribution
current_samples = pd.DataFrame(columns=['user id', 'tweets per day'])
for tpd in current_tpds['tweets per day'].unique():
    tmp_df = current_tpds[current_tpds['tweets per day']==tpd]
    frac = float(dist_df.loc[dist_df['tweets per day']==tpd]['percentage'].iloc[0])
    if int(n_depression*frac)<len(tmp_df):
        sampled_tmp_df = tmp_df.sample(n=int(n_depression*frac), random_state=42)
    else:
        sampled_tmp_df = tmp_df
    current_samples = current_samples._append(sampled_tmp_df, ignore_index=True)

present_tpd = current_samples['tweets per day'].unique().tolist()
all_tpd = dist_df['tweets per day'].unique().tolist()
diff_tpd = list(set(present_tpd).symmetric_difference(set(all_tpd)))
dist_df = dist_df.sort_values('percentage', ascending=False) #sort percentages here

#fill up remaining spots with random users from rs cohort not yet present in sampled users starting with most occuring tpd class in filtered user list (all tpd not already in sampled list)
needed_tpd = []
counter = n_depression-len(current_samples['user id'].unique())
if counter > 0:
    for tpd in dist_df['tweets per day'].unique():
        if tpd in diff_tpd and counter > 0:
            tmp_df = current_tpds[current_tpds['tweets per day']==tpd]
            #make sure to not sample a user again
            sampled_tmp_df = tmp_df.sample(n=1, random_state=42)
            if current_samples['user id'].eq(sampled_tmp_df['user id'].tolist()[0]).any():
                unique = False
                stop = len(current_tpds['user id'].unique())
                while (not unique) or (stop>0):
                    sampled_tmp_df = tmp_df.sample(n=1, random_state=42)
                    if not current_samples['user id'].eq(sampled_tmp_df['user id'].tolist()[0]).any():
                        unique = True
                    stop = stop - 1

            current_samples = current_samples._append(sampled_tmp_df, ignore_index=True)
            counter = counter - 1
print(counter)
print(len(current_samples['user id'].unique()))
#if still space left, fill again, this time not caring if tpd is already in final list with samples from all users in rs cohort, with disregard to filtered users

current_user_list = list(set(st.user_id.unique()).symmetric_difference(set(current_samples['user id'].unique())))

avg_dist = {}
current_tpds = tpd_pairs[tpd_pairs['user id'].isin(set(current_user_list))]
for avg in current_tpds['tweets per day'].unique():
    tmp_dist = float((current_tpds[current_tpds['tweets per day']==avg].count()/len(current_tpds)).iloc[0])
    avg_dist.update({str(avg): str(tmp_dist)})

dist_df = pd.DataFrame(avg_dist.items(), columns=['tweets per day', 'percentage'])
dist_df = dist_df.astype({'tweets per day': 'int32'})
dist_df = dist_df.astype({'percentage': 'float'})
dist_df = dist_df.sort_values(by=['tweets per day'])

for tpd in current_tpds['tweets per day'].unique():
    tmp_df = current_tpds[current_tpds['tweets per day']==tpd]
    frac = float(dist_df.loc[dist_df['tweets per day']==tpd]['percentage'].iloc[0])
    if int(n_depression*frac)<len(tmp_df):
        sampled_tmp_df = tmp_df.sample(n=int(n_depression*frac), random_state=42)
    else:
        sampled_tmp_df = tmp_df
    current_samples = current_samples._append(sampled_tmp_df, ignore_index=True)

present_tpd = current_samples['tweets per day'].unique().tolist()
all_tpd = dist_df['tweets per day'].unique().tolist()
diff_tpd = list(set(present_tpd).symmetric_difference(set(all_tpd)))
dist_df = dist_df.sort_values('percentage', ascending=False) #sort percentages here

needed_tpd = []
counter = n_depression-len(current_samples['user id'].unique())
while counter > 0:
    for tpd in dist_df['tweets per day'].unique():
        if tpd in diff_tpd and counter > 0:
            tmp_df = current_tpds[current_tpds['tweets per day']==tpd]
            #make sure to not sample a user again
            sampled_tmp_df = tmp_df.sample(n=1, random_state=42)
            if current_samples['user id'].eq(sampled_tmp_df['user id'].tolist()[0]).any():
                unique = False
                stop = len(current_tpds['user id'].unique())
                while (not unique) or (stop>0):
                    sampled_tmp_df = tmp_df.sample(n=1, random_state=42)
                    if not current_samples['user id'].eq(sampled_tmp_df['user id'].tolist()[0]).any():
                        unique = True
                    stop = stop - 1

            current_samples = current_samples._append(sampled_tmp_df, ignore_index=True)
            counter = counter - 1

print(len(current_samples['user id'].unique()))

Unnamed: 0                                                       108
min_tpd                                                           10
max_days_bt                                                        1
min_days_active                                                   30
user_list          ['uR0011', 'uR0014', 'uR0015', 'uR0022', 'uR00...
Name: 948, dtype: object
16
587
1156


In [3]:
# create dataset as df list with uniform dfs
data = {}
labels = []
data_list = []

n_days = 0
for user in dt.user_id.unique():
    tmp = dt[dt['user_id'] == user]
    tmp_days = len(tmp['created_at'].map(lambda datetime: datetime.date()).unique())  #number of recorded days
    if tmp_days > n_days:
        n_days = tmp_days

for user in current_samples['user id'].unique():
    tmp = st[st['user_id'] == user]
    tmp_days = len(tmp['created_at'].map(lambda datetime: datetime.date()).unique())  #number of recorded days
    if tmp_days > n_days:
        n_days = tmp_days

for user in dt.user_id.unique():
    tmp = dt[dt['user_id'] == user]
    tmp_days = len(tmp['created_at'].map(lambda datetime: datetime.date()).unique())  #number of recorded days
    timestamps = tmp['created_at'].iloc[::-1]  #sort dates in ascending order
    matrix = np.full((n_days, 24), 0)  #create matrix for one individual (#recorded days*24)

    last_date = None
    for d in range(0, tmp_days):
        current_date = timestamps.iloc[d].date()
        if current_date == last_date:
            continue  # if the current date has already been looked at, move on

        cur_tmp = tmp[tmp['created_at'].map(lambda datetime: datetime.date()) == timestamps.iloc[
            d].date()]  #get list of timestamps where the date is the currently observed date regardless of time
        for e in cur_tmp['created_at']:
            hour_i = e.hour  # user hour of timestamp as index in matrix
            matrix[d, hour_i] = matrix[d, hour_i] + 1  # move through matrix and add tweets per hour

        last_date = current_date

    data.update({'' + user + '': matrix})
    data_list.append(pd.DataFrame(matrix))
    labels.append(1)

for user in current_samples['user id'].unique():
    tmp = st[st['user_id'] == user]
    tmp_days = len(tmp['created_at'].map(lambda datetime: datetime.date()).unique())  #number of recorded days
    timestamps = tmp['created_at'].iloc[::-1]  #sort dates in ascending order
    matrix = np.full((n_days, 24), 0)  #create matrix for one individual (#recorded days*24)

    last_date = None
    for d in range(0, tmp_days):
        current_date = timestamps.iloc[d].date()
        if current_date == last_date:
            continue  # if the current date has already been looked at, move on

        cur_tmp = tmp[tmp['created_at'].map(lambda datetime: datetime.date()) == timestamps.iloc[
            d].date()]  #get list of timestamps where the date is the currently observed date regardless of time
        for e in cur_tmp['created_at']:
            hour_i = e.hour  # user hour of timestamp as index in matrix
            matrix[d, hour_i] = matrix[d, hour_i] + 1  # move through matrix and add tweets per hour

        last_date = current_date

    data.update({'' + user + '': matrix})
    data_list.append(pd.DataFrame(matrix))
    labels.append(0)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_list, labels, test_size=0.2)
X_train = np.stack([df.to_numpy() for df in X_train]).astype('float32')
X_test = np.stack([df.to_numpy() for df in X_test]).astype('float32')

train_max = X_train.max(axis=(0, 1))
train_min = X_train.min(axis=(0, 1))

X_train = (X_train - train_min) / (train_max - train_min)
X_test = (X_test - train_min) / (train_max - train_min)
y_train = np.array(y_train, dtype='float32')
y_test = np.array(y_test, dtype='float32')

In [4]:
from tensorboard.plugins.hparams import api as hp
import uuid

from keras import Sequential
from keras import losses, metrics
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import layers

HP_LSTM_UNITS = hp.HParam('lstm units', hp.Discrete([20,30,40,50,60,70]))
HP_OPTIMIZERS = hp.HParam('optimizers', hp.Discrete(['Adam', 'Adagrad']))
HP_LSTM_ACTIVATION = hp.HParam('activation', hp.Discrete(['relu', 'sigmoid', 'tanh', 'selu', 'elu', 'exponential']))
HP_LSTM_REC_ACTIVATION = hp.HParam('reccurring activation', hp.Discrete(['relu', 'sigmoid', 'tanh', 'selu', 'elu', 'exponential']))
HP_LSTM_DROPOUT = hp.HParam('dropout', hp.Discrete([0.,0.1,0.2]))

METRIC_BIN_ACC = 'binary accuracy'
METRIC_AUC = 'AUC'
METRIC_PRECISION = 'precision'
METRIC_RECALL = 'recall'
METRIC_F1 = 'f1'
METRIC_TN = 'true negatives'
METRIC_TP = 'true positives'
METRIC_FN = 'false negatives'
METRIC_FP = 'false positives'
METRIC_TRAIN_TIME = 'train time'
METRIC_TEST_TIME = 'test time'

with tf.summary.create_file_writer('final_log_user_group_948/classifier_test_lstm_sigmoid_all_activations_full_metrics_3').as_default():
  hp.hparams_config(
    hparams=[HP_LSTM_UNITS,
             HP_OPTIMIZERS,
             HP_LSTM_ACTIVATION,
             HP_LSTM_REC_ACTIVATION,
             HP_LSTM_DROPOUT],
    metrics=[(hp.Metric(METRIC_BIN_ACC, display_name='binary accuracy')),
             (hp.Metric(METRIC_AUC, display_name='auc')),
             (hp.Metric(METRIC_PRECISION, display_name='precision')),
             (hp.Metric(METRIC_RECALL, display_name='recall')),
             (hp.Metric(METRIC_F1, display_name='f1')),
             (hp.Metric(METRIC_TN, display_name='true negatives')),
             (hp.Metric(METRIC_TP, display_name='true positives')),
             (hp.Metric(METRIC_FN, display_name='false negatives')),
             (hp.Metric(METRIC_FP, display_name='false positives')),
             (hp.Metric(METRIC_TRAIN_TIME, display_name='train time')),
             (hp.Metric(METRIC_TEST_TIME, display_name='test time'))],
  )


#writer = tf.summary.create_file_writer('log/classifier_test_test')

lstm_units_list = [30,40,50,60,70]
optimizer_list = ['Adam', 'Adagrad']
activation_list = ['sigmoid', 'tanh']
rec_activation_list = ['sigmoid', 'tanh']
##activation_list = ['sigmoid', 'tanh']
#rec_activation_list = ['sigmoid', 'tanh']
dropout_list = [0,0.1,0.2]


for u in lstm_units_list:
    for o in optimizer_list:
        for a in activation_list:
            for r in rec_activation_list:
                for d in dropout_list:
                    id = uuid.uuid4()
                    hparams = {HP_LSTM_UNITS: u, HP_OPTIMIZERS: o, HP_LSTM_ACTIVATION: a, HP_LSTM_REC_ACTIVATION: r, HP_LSTM_DROPOUT: d}
                    writer = tf.summary.create_file_writer('final_log_user_group_948/classifier_test_lstm_sigmoid_all_activations_full_metrics_3/'+str(id))
                    with writer.as_default():
                        callbacks = [hp.KerasCallback('final_log_user_group_948/classifier_test_lstm_sigmoid_all_activations_full_metrics_3/'+str(id), hparams),
                                     tf.keras.callbacks.EarlyStopping(
                                        monitor="loss",
                                        min_delta=0.00005,
                                        patience=10,
                                        verbose=1,
                                        mode="auto",
                                        baseline=None,
                                        restore_best_weights=False,
                                        start_from_epoch=10,
                                    ),]

                        model = Sequential()
                        model.add(layers.Input(X_train.shape[1:]))
                        model.add(layers.LSTM(u, activation=a, recurrent_activation=r, dropout=d))
                        model.add(layers.Dense(1, activation="sigmoid"))
                        model.compile(optimizer=o, loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy(),metrics.AUC(),metrics.Precision(),metrics.Recall(),metrics.F1Score(), metrics.TrueNegatives(), metrics.TruePositives(), metrics.FalseNegatives(), metrics.FalsePositives()])

                        start = time.perf_counter()
                        model.fit(X_train, y_train, epochs=25, callbacks=callbacks)
                        end = time.perf_counter()
                        train_time = end - start

                        start = time.perf_counter()
                        metric_results = model.evaluate(X_test, y_test)
                        end = time.perf_counter()
                        test_time = end - start

                        if a != 'relu' and np.isnan(metric_results[0]):
                            print(metric_results)

                        hp.hparams(hparams)
                        tf.summary.scalar(METRIC_BIN_ACC,metric_results[0], step=1)
                        tf.summary.scalar(METRIC_AUC,metric_results[1], step=1)
                        tf.summary.scalar(METRIC_PRECISION,metric_results[2], step=1)
                        tf.summary.scalar(METRIC_RECALL,metric_results[3], step=1)
                        tf.summary.scalar(METRIC_F1,metric_results[4], step=1)
                        tf.summary.scalar(METRIC_TN,metric_results[5][0], step=1)
                        tf.summary.scalar(METRIC_TP,metric_results[6], step=1)
                        tf.summary.scalar(METRIC_FN,metric_results[7], step=1)
                        tf.summary.scalar(METRIC_FP,metric_results[8], step=1)
                        tf.summary.scalar(METRIC_TRAIN_TIME, train_time, step=1)
                        tf.summary.scalar(METRIC_TEST_TIME, test_time, step=1)

                        keras.backend.clear_session()


Epoch 1/25
44/44 [==============================] - 13s 266ms/step - loss: 0.7365 - binary_accuracy: 0.4648 - auc: 0.4934 - precision: 0.3414 - recall: 0.6159 - f1_score: 0.5080 - true_negatives: 359.0000 - true_positives: 295.0000 - false_negatives: 184.0000 - false_positives: 569.0000
Epoch 2/25
44/44 [==============================] - 12s 263ms/step - loss: 0.6411 - binary_accuracy: 0.6596 - auc: 0.5275 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.5080 - true_negatives: 928.0000 - true_positives: 0.0000e+00 - false_negatives: 479.0000 - false_positives: 0.0000e+00
Epoch 3/25
44/44 [==============================] - 12s 266ms/step - loss: 0.6419 - binary_accuracy: 0.6596 - auc: 0.4719 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.5080 - true_negatives: 928.0000 - true_positives: 0.0000e+00 - false_negatives: 479.0000 - false_positives: 0.0000e+00
Epoch 4/25
44/44 [==============================] - 12s 266ms/step - loss: 0.6417 - binary_accuracy: 0.6596 - au

# 1:2 ratio

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import layers

#depression tweets
dt = pd.read_csv("data/depression_tweets.tsv", sep="\t", index_col="tweet_id", parse_dates=[2])
#dt["created_at"] = pd.to_datetime(dt["created_at"])
n_depression = len(dt['user_id'].unique())*2
print(n_depression)

#sample tweets
st = pd.read_csv("data/sample_tweets.tsv", sep="\t", index_col="tweet_id", parse_dates=[2])
#st["created_at"] = pd.to_datetime(st["created_at"])

#filtered users
fu = pd.read_csv('filtered_user_lists.csv')
fu_tbt = pd.read_csv('filtered_user_lists_only_tpd_tbt.csv')
fu_maxtime = pd.read_csv('filtered_user_lists_only_tpd_maxtime.csv')
#tpd pairs
tpd_pairs = pd.read_csv('pairs_tpd_users.csv')
filtered_users = pd.concat([fu, fu_tbt, fu_maxtime], ignore_index=True)
j = 948
current = filtered_users.iloc[j]
print(current)
current_user_list = eval(current.user_list)

#get distribution of current filtered list
avg_dist = {}
current_tpds = tpd_pairs[tpd_pairs['user id'].isin(set(current_user_list))]
for avg in current_tpds['tweets per day'].unique():
    tmp_dist = float((current_tpds[current_tpds['tweets per day'] == avg].count() / len(current_tpds)).iloc[0])
    avg_dist.update({str(avg): str(tmp_dist)})

dist_df = pd.DataFrame(avg_dist.items(), columns=['tweets per day', 'percentage'])
dist_df = dist_df.astype({'tweets per day': 'int32'})
dist_df = dist_df.astype({'percentage': 'float'})
dist_df = dist_df.sort_values(by=['tweets per day'])

#downsample current user list based on distribution
current_samples = pd.DataFrame(columns=['user id', 'tweets per day'])
for tpd in current_tpds['tweets per day'].unique():
    tmp_df = current_tpds[current_tpds['tweets per day'] == tpd]
    frac = float(dist_df.loc[dist_df['tweets per day'] == tpd]['percentage'].iloc[0])
    if int(n_depression * frac) < len(tmp_df):
        sampled_tmp_df = tmp_df.sample(n=int(n_depression * frac), random_state=42)
    else:
        sampled_tmp_df = tmp_df
    current_samples = current_samples._append(sampled_tmp_df, ignore_index=True)

present_tpd = current_samples['tweets per day'].unique().tolist()
all_tpd = dist_df['tweets per day'].unique().tolist()
diff_tpd = list(set(present_tpd).symmetric_difference(set(all_tpd)))
dist_df = dist_df.sort_values('percentage', ascending=False)  #sort percentages here

#fill up remaining spots with random users from rs cohort not yet present in sampled users starting with most occuring tpd class in filtered user list (all tpd not already in sampled list)
needed_tpd = []
counter = n_depression - len(current_samples['user id'].unique())
if counter > 0:
    for tpd in dist_df['tweets per day'].unique():
        if tpd in diff_tpd and counter > 0:
            tmp_df = current_tpds[current_tpds['tweets per day'] == tpd]
            #make sure to not sample a user again
            sampled_tmp_df = tmp_df.sample(n=1, random_state=42)
            if current_samples['user id'].eq(sampled_tmp_df['user id'].tolist()[0]).any():
                unique = False
                stop = len(current_tpds['user id'].unique())
                while (not unique) or (stop > 0):
                    sampled_tmp_df = tmp_df.sample(n=1, random_state=42)
                    if not current_samples['user id'].eq(sampled_tmp_df['user id'].tolist()[0]).any():
                        unique = True
                    stop = stop - 1

            current_samples = current_samples._append(sampled_tmp_df, ignore_index=True)
            counter = counter - 1
print(counter)
print(len(current_samples['user id'].unique()))
#if still space left, fill again, this time not caring if tpd is already in final list with samples from all users in rs cohort, with disregard to filtered users

current_user_list = list(set(st.user_id.unique()).symmetric_difference(set(current_samples['user id'].unique())))

avg_dist = {}
current_tpds = tpd_pairs[tpd_pairs['user id'].isin(set(current_user_list))]
for avg in current_tpds['tweets per day'].unique():
    tmp_dist = float((current_tpds[current_tpds['tweets per day'] == avg].count() / len(current_tpds)).iloc[0])
    avg_dist.update({str(avg): str(tmp_dist)})

dist_df = pd.DataFrame(avg_dist.items(), columns=['tweets per day', 'percentage'])
dist_df = dist_df.astype({'tweets per day': 'int32'})
dist_df = dist_df.astype({'percentage': 'float'})
dist_df = dist_df.sort_values(by=['tweets per day'])

for tpd in current_tpds['tweets per day'].unique():
    tmp_df = current_tpds[current_tpds['tweets per day'] == tpd]
    frac = float(dist_df.loc[dist_df['tweets per day'] == tpd]['percentage'].iloc[0])
    if int(n_depression * frac) < len(tmp_df):
        sampled_tmp_df = tmp_df.sample(n=int(n_depression * frac), random_state=42)
    else:
        sampled_tmp_df = tmp_df
    current_samples = current_samples._append(sampled_tmp_df, ignore_index=True)

present_tpd = current_samples['tweets per day'].unique().tolist()
all_tpd = dist_df['tweets per day'].unique().tolist()
diff_tpd = list(set(present_tpd).symmetric_difference(set(all_tpd)))
dist_df = dist_df.sort_values('percentage', ascending=False)  #sort percentages here

needed_tpd = []
counter = n_depression - len(current_samples['user id'].unique())
while counter > 0:
    for tpd in dist_df['tweets per day'].unique():
        if tpd in diff_tpd and counter > 0:
            tmp_df = current_tpds[current_tpds['tweets per day'] == tpd]
            #make sure to not sample a user again
            sampled_tmp_df = tmp_df.sample(n=1, random_state=42)
            if current_samples['user id'].eq(sampled_tmp_df['user id'].tolist()[0]).any():
                unique = False
                stop = len(current_tpds['user id'].unique())
                while (not unique) or (stop > 0):
                    sampled_tmp_df = tmp_df.sample(n=1, random_state=42)
                    if not current_samples['user id'].eq(sampled_tmp_df['user id'].tolist()[0]).any():
                        unique = True
                    stop = stop - 1

            current_samples = current_samples._append(sampled_tmp_df, ignore_index=True)
            counter = counter - 1

print(len(current_samples['user id'].unique()))
# create dataset as df list with uniform dfs
data = {}
labels = []
data_list = []

n_days = 0
for user in dt.user_id.unique():
    tmp = dt[dt['user_id'] == user]
    tmp_days = len(tmp['created_at'].map(lambda datetime: datetime.date()).unique())  #number of recorded days
    if tmp_days > n_days:
        n_days = tmp_days

for user in current_samples['user id'].unique():
    tmp = st[st['user_id'] == user]
    tmp_days = len(tmp['created_at'].map(lambda datetime: datetime.date()).unique())  #number of recorded days
    if tmp_days > n_days:
        n_days = tmp_days

for user in dt.user_id.unique():
    tmp = dt[dt['user_id'] == user]
    tmp_days = len(tmp['created_at'].map(lambda datetime: datetime.date()).unique())  #number of recorded days
    timestamps = tmp['created_at'].iloc[::-1]  #sort dates in ascending order
    matrix = np.full((n_days, 24), 0)  #create matrix for one individual (#recorded days*24)

    last_date = None
    for d in range(0, tmp_days):
        current_date = timestamps.iloc[d].date()
        if current_date == last_date:
            continue  # if the current date has already been looked at, move on

        cur_tmp = tmp[tmp['created_at'].map(lambda datetime: datetime.date()) == timestamps.iloc[
            d].date()]  #get list of timestamps where the date is the currently observed date regardless of time
        for e in cur_tmp['created_at']:
            hour_i = e.hour  # user hour of timestamp as index in matrix
            matrix[d, hour_i] = matrix[d, hour_i] + 1  # move through matrix and add tweets per hour

        last_date = current_date

    data.update({'' + user + '': matrix})
    data_list.append(pd.DataFrame(matrix))
    labels.append(1)

for user in current_samples['user id'].unique():
    tmp = st[st['user_id'] == user]
    tmp_days = len(tmp['created_at'].map(lambda datetime: datetime.date()).unique())  #number of recorded days
    timestamps = tmp['created_at'].iloc[::-1]  #sort dates in ascending order
    matrix = np.full((n_days, 24), 0)  #create matrix for one individual (#recorded days*24)

    last_date = None
    for d in range(0, tmp_days):
        current_date = timestamps.iloc[d].date()
        if current_date == last_date:
            continue  # if the current date has already been looked at, move on

        cur_tmp = tmp[tmp['created_at'].map(lambda datetime: datetime.date()) == timestamps.iloc[
            d].date()]  #get list of timestamps where the date is the currently observed date regardless of time
        for e in cur_tmp['created_at']:
            hour_i = e.hour  # user hour of timestamp as index in matrix
            matrix[d, hour_i] = matrix[d, hour_i] + 1  # move through matrix and add tweets per hour

        last_date = current_date

    data.update({'' + user + '': matrix})
    data_list.append(pd.DataFrame(matrix))
    labels.append(0)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_list, labels, test_size=0.2)
X_train = np.stack([df.to_numpy() for df in X_train]).astype('float32')
X_test = np.stack([df.to_numpy() for df in X_test]).astype('float32')

train_max = X_train.max(axis=(0, 1))
train_min = X_train.min(axis=(0, 1))

X_train = (X_train - train_min) / (train_max - train_min)
X_test = (X_test - train_min) / (train_max - train_min)
y_train = np.array(y_train, dtype='float32')
y_test = np.array(y_test, dtype='float32')

1206
Unnamed: 0                                                       108
min_tpd                                                           10
max_days_bt                                                        1
min_days_active                                                   30
user_list          ['uR0011', 'uR0014', 'uR0015', 'uR0022', 'uR00...
Name: 948, dtype: object
96
1110
2273


In [ ]:
from tensorboard.plugins.hparams import api as hp
import uuid

from keras import Sequential
from keras import losses, metrics
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import layers
import time

HP_LSTM_UNITS = hp.HParam('lstm units', hp.Discrete([20,30,40,50,60,70]))
HP_OPTIMIZERS = hp.HParam('optimizers', hp.Discrete(['Adam', 'Adagrad']))
HP_LSTM_ACTIVATION = hp.HParam('activation', hp.Discrete(['relu', 'sigmoid', 'tanh', 'selu', 'elu', 'exponential']))
HP_LSTM_REC_ACTIVATION = hp.HParam('reccurring activation', hp.Discrete(['relu', 'sigmoid', 'tanh', 'selu', 'elu', 'exponential']))
HP_LSTM_DROPOUT = hp.HParam('dropout', hp.Discrete([0.,0.1,0.2]))

METRIC_BIN_ACC = 'binary accuracy'
METRIC_AUC = 'AUC'
METRIC_PRECISION = 'precision'
METRIC_RECALL = 'recall'
METRIC_F1 = 'f1'
METRIC_TN = 'true negatives'
METRIC_TP = 'true positives'
METRIC_FN = 'false negatives'
METRIC_FP = 'false positives'
METRIC_TRAIN_TIME = 'train time'
METRIC_TEST_TIME = 'test time'

with tf.summary.create_file_writer('final_log_doubleRS_user_group_948/classifier_test_lstm_sigmoid_all_activations_full_metrics').as_default():
  hp.hparams_config(
    hparams=[HP_LSTM_UNITS,
             HP_OPTIMIZERS,
             HP_LSTM_ACTIVATION,
             HP_LSTM_REC_ACTIVATION,
             HP_LSTM_DROPOUT],
    metrics=[(hp.Metric(METRIC_BIN_ACC, display_name='binary accuracy')),
             (hp.Metric(METRIC_AUC, display_name='auc')),
             (hp.Metric(METRIC_PRECISION, display_name='precision')),
             (hp.Metric(METRIC_RECALL, display_name='recall')),
             (hp.Metric(METRIC_F1, display_name='f1')),
             (hp.Metric(METRIC_TN, display_name='true negatives')),
             (hp.Metric(METRIC_TP, display_name='true positives')),
             (hp.Metric(METRIC_FN, display_name='false negatives')),
             (hp.Metric(METRIC_FP, display_name='false positives')),
             (hp.Metric(METRIC_TRAIN_TIME, display_name='train time')),
             (hp.Metric(METRIC_TEST_TIME, display_name='test time'))],
  )


#writer = tf.summary.create_file_writer('log/classifier_test_test')

lstm_units_list = [20,30,40,50,60,70]
optimizer_list = ['Adam', 'Adagrad']
activation_list = ['sigmoid', 'tanh']
rec_activation_list = ['sigmoid', 'tanh']
##activation_list = ['sigmoid', 'tanh']
#rec_activation_list = ['sigmoid', 'tanh']
dropout_list = [0,0.1,0.2]


for u in lstm_units_list:
    for o in optimizer_list:
        for a in activation_list:
            for r in rec_activation_list:
                for d in dropout_list:
                    id = uuid.uuid4()
                    hparams = {HP_LSTM_UNITS: u, HP_OPTIMIZERS: o, HP_LSTM_ACTIVATION: a, HP_LSTM_REC_ACTIVATION: r, HP_LSTM_DROPOUT: d}
                    writer = tf.summary.create_file_writer('final_log_doubleRS_user_group_948/classifier_test_lstm_sigmoid_all_activations_full_metrics/'+str(id))
                    with writer.as_default():
                        callbacks = [hp.KerasCallback('final_log_doubleRS_user_group_948/classifier_test_lstm_sigmoid_all_activations_full_metrics/'+str(id), hparams),
                                     tf.keras.callbacks.EarlyStopping(
                                        monitor="loss",
                                        min_delta=0.00005,
                                        patience=10,
                                        verbose=1,
                                        mode="auto",
                                        baseline=None,
                                        restore_best_weights=False,
                                        start_from_epoch=10,
                                    ),]

                        model = Sequential()
                        model.add(layers.Input(X_train.shape[1:]))
                        model.add(layers.LSTM(u, activation=a, recurrent_activation=r, dropout=d))
                        model.add(layers.Dense(1, activation="sigmoid"))
                        model.compile(optimizer=o, loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy(),metrics.AUC(),metrics.Precision(),metrics.Recall(),metrics.F1Score(), metrics.TrueNegatives(), metrics.TruePositives(), metrics.FalseNegatives(), metrics.FalsePositives()])

                        start = time.perf_counter()
                        model.fit(X_train, y_train, epochs=25, callbacks=callbacks)
                        end = time.perf_counter()
                        train_time = end - start

                        start = time.perf_counter()
                        metric_results = model.evaluate(X_test, y_test)
                        end = time.perf_counter()
                        test_time = end - start

                        if a != 'relu' and np.isnan(metric_results[0]):
                            print(metric_results)

                        hp.hparams(hparams)
                        tf.summary.scalar(METRIC_BIN_ACC,metric_results[0], step=1)
                        tf.summary.scalar(METRIC_AUC,metric_results[1], step=1)
                        tf.summary.scalar(METRIC_PRECISION,metric_results[2], step=1)
                        tf.summary.scalar(METRIC_RECALL,metric_results[3], step=1)
                        tf.summary.scalar(METRIC_F1,metric_results[4], step=1)
                        tf.summary.scalar(METRIC_TN,metric_results[5][0], step=1)
                        tf.summary.scalar(METRIC_TP,metric_results[6], step=1)
                        tf.summary.scalar(METRIC_FN,metric_results[7], step=1)
                        tf.summary.scalar(METRIC_FP,metric_results[8], step=1)
                        tf.summary.scalar(METRIC_TRAIN_TIME, train_time, step=1)
                        tf.summary.scalar(METRIC_TEST_TIME, test_time, step=1)

                        keras.backend.clear_session()


Epoch 1/25
72/72 [==============================] - 18s 214ms/step - loss: 0.5769 - binary_accuracy: 0.7513 - auc_1: 0.5064 - precision_1: 0.2344 - recall_1: 0.0957 - f1_score: 0.3394 - true_negatives_1: 1683.0000 - true_positives_1: 45.0000 - false_negatives_1: 425.0000 - false_positives_1: 147.0000
Epoch 2/25
72/72 [==============================] - 15s 213ms/step - loss: 0.5070 - binary_accuracy: 0.7957 - auc_1: 0.4809 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - f1_score: 0.3394 - true_negatives_1: 1830.0000 - true_positives_1: 0.0000e+00 - false_negatives_1: 470.0000 - false_positives_1: 0.0000e+00
Epoch 3/25
72/72 [==============================] - 17s 230ms/step - loss: 0.5068 - binary_accuracy: 0.7957 - auc_1: 0.4856 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - f1_score: 0.3394 - true_negatives_1: 1830.0000 - true_positives_1: 0.0000e+00 - false_negatives_1: 470.0000 - false_positives_1: 0.0000e+00
Epoch 4/25
72/72 [==============================] - 16s 217ms/step -

# full

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import layers

#depression tweets
dt = pd.read_csv("data/depression_tweets.tsv", sep="\t", index_col="tweet_id", parse_dates=[2])
#dt["created_at"] = pd.to_datetime(dt["created_at"])
n_depression = len(dt['user_id'].unique())
print(n_depression)

#sample tweets
st = pd.read_csv("data/sample_tweets.tsv", sep="\t", index_col="tweet_id", parse_dates=[2])
#st["created_at"] = pd.to_datetime(st["created_at"])


# create dataset as df list with uniform dfs
data = {}
labels = []
data_list = []

n_days = 0
for user in dt.user_id.unique():
    tmp = dt[dt['user_id'] == user]
    tmp_days = len(tmp['created_at'].map(lambda datetime: datetime.date()).unique())  #number of recorded days
    if tmp_days > n_days:
        n_days = tmp_days

for user in st['user_id'].unique():
    tmp = st[st['user_id'] == user]
    tmp_days = len(tmp['created_at'].map(lambda datetime: datetime.date()).unique())  #number of recorded days
    if tmp_days > n_days:
        n_days = tmp_days

for user in dt.user_id.unique():
    tmp = dt[dt['user_id'] == user]
    tmp_days = len(tmp['created_at'].map(lambda datetime: datetime.date()).unique())  #number of recorded days
    timestamps = tmp['created_at'].iloc[::-1]  #sort dates in ascending order
    matrix = np.full((n_days, 24), 0)  #create matrix for one individual (#recorded days*24)

    last_date = None
    for d in range(0, tmp_days):
        current_date = timestamps.iloc[d].date()
        if current_date == last_date:
            continue  # if the current date has already been looked at, move on

        cur_tmp = tmp[tmp['created_at'].map(lambda datetime: datetime.date()) == timestamps.iloc[
            d].date()]  #get list of timestamps where the date is the currently observed date regardless of time
        for e in cur_tmp['created_at']:
            hour_i = e.hour  # user hour of timestamp as index in matrix
            matrix[d, hour_i] = matrix[d, hour_i] + 1  # move through matrix and add tweets per hour

        last_date = current_date

    data.update({'' + user + '': matrix})
    data_list.append(pd.DataFrame(matrix))
    labels.append(1)

for user in st['user_id'].unique():
    tmp = st[st['user_id'] == user]
    tmp_days = len(tmp['created_at'].map(lambda datetime: datetime.date()).unique())  #number of recorded days
    timestamps = tmp['created_at'].iloc[::-1]  #sort dates in ascending order
    matrix = np.full((n_days, 24), 0)  #create matrix for one individual (#recorded days*24)

    last_date = None
    for d in range(0, tmp_days):
        current_date = timestamps.iloc[d].date()
        if current_date == last_date:
            continue  # if the current date has already been looked at, move on

        cur_tmp = tmp[tmp['created_at'].map(lambda datetime: datetime.date()) == timestamps.iloc[
            d].date()]  #get list of timestamps where the date is the currently observed date regardless of time
        for e in cur_tmp['created_at']:
            hour_i = e.hour  # user hour of timestamp as index in matrix
            matrix[d, hour_i] = matrix[d, hour_i] + 1  # move through matrix and add tweets per hour

        last_date = current_date

    data.update({'' + user + '': matrix})
    data_list.append(pd.DataFrame(matrix))
    labels.append(0)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_list, labels, test_size=0.2)
X_train = np.stack([df.to_numpy() for df in X_train]).astype('float32')
X_test = np.stack([df.to_numpy() for df in X_test]).astype('float32')

train_max = X_train.max(axis=(0, 1))
train_min = X_train.min(axis=(0, 1))

X_train = (X_train - train_min) / (train_max - train_min)
X_test = (X_test - train_min) / (train_max - train_min)
y_train = np.array(y_train, dtype='float32')
y_test = np.array(y_test, dtype='float32')

603


In [ ]:
from tensorboard.plugins.hparams import api as hp
import uuid

from keras import Sequential
from keras import losses, metrics
import time

HP_LSTM_UNITS = hp.HParam('lstm units', hp.Discrete([20,30,40,50,60,70]))
HP_OPTIMIZERS = hp.HParam('optimizers', hp.Discrete(['Adam', 'Adagrad']))
HP_LSTM_ACTIVATION = hp.HParam('activation', hp.Discrete(['relu', 'sigmoid', 'tanh', 'selu', 'elu', 'exponential']))
HP_LSTM_REC_ACTIVATION = hp.HParam('reccurring activation', hp.Discrete(['relu', 'sigmoid', 'tanh', 'selu', 'elu', 'exponential']))
HP_LSTM_DROPOUT = hp.HParam('dropout', hp.Discrete([0.,0.1,0.2]))

METRIC_BIN_ACC = 'binary accuracy'
METRIC_AUC = 'AUC'
METRIC_PRECISION = 'precision'
METRIC_RECALL = 'recall'
METRIC_F1 = 'f1'
METRIC_TN = 'true negatives'
METRIC_TP = 'true positives'
METRIC_FN = 'false negatives'
METRIC_FP = 'false positives'
METRIC_TRAIN_TIME = 'train time'
METRIC_TEST_TIME = 'test time'

with tf.summary.create_file_writer('final_log_fullRS/classifier_test_lstm_sigmoid_all_activations_full_metrics_2').as_default():
  hp.hparams_config(
    hparams=[HP_LSTM_UNITS,
             HP_OPTIMIZERS,
             HP_LSTM_ACTIVATION,
             HP_LSTM_REC_ACTIVATION,
             HP_LSTM_DROPOUT],
    metrics=[(hp.Metric(METRIC_BIN_ACC, display_name='binary accuracy')),
             (hp.Metric(METRIC_AUC, display_name='auc')),
             (hp.Metric(METRIC_PRECISION, display_name='precision')),
             (hp.Metric(METRIC_RECALL, display_name='recall')),
             (hp.Metric(METRIC_F1, display_name='f1')),
             (hp.Metric(METRIC_TN, display_name='true negatives')),
             (hp.Metric(METRIC_TP, display_name='true positives')),
             (hp.Metric(METRIC_FN, display_name='false negatives')),
             (hp.Metric(METRIC_FP, display_name='false positives')),
             (hp.Metric(METRIC_TRAIN_TIME, display_name='train time')),
             (hp.Metric(METRIC_TEST_TIME, display_name='test time'))],
  )


#writer = tf.summary.create_file_writer('log/classifier_test_test')

lstm_units_list = [70]
optimizer_list = ['Adam', 'Adagrad']
activation_list = ['sigmoid', 'tanh']
rec_activation_list = ['sigmoid', 'tanh']
##activation_list = ['sigmoid', 'tanh']
#rec_activation_list = ['sigmoid', 'tanh']
dropout_list = [0,0.1,0.2]


for u in lstm_units_list:
    for o in optimizer_list:
        for a in activation_list:
            for r in rec_activation_list:
                for d in dropout_list:
                    id = uuid.uuid4()
                    hparams = {HP_LSTM_UNITS: u, HP_OPTIMIZERS: o, HP_LSTM_ACTIVATION: a, HP_LSTM_REC_ACTIVATION: r, HP_LSTM_DROPOUT: d}
                    writer = tf.summary.create_file_writer('final_log_fullRS/classifier_test_lstm_sigmoid_all_activations_full_metrics_2/'+str(id))
                    with writer.as_default():
                        callbacks = [hp.KerasCallback('final_log_fullRS/classifier_test_lstm_sigmoid_all_activations_full_metrics_2/'+str(id), hparams),
                                     tf.keras.callbacks.EarlyStopping(
                                        monitor="loss",
                                        min_delta=0.00005,
                                        patience=10,
                                        verbose=1,
                                        mode="auto",
                                        baseline=None,
                                        restore_best_weights=False,
                                        start_from_epoch=10,
                                    ),]

                        model = Sequential()
                        model.add(layers.Input(X_train.shape[1:]))
                        model.add(layers.LSTM(u, activation=a, recurrent_activation=r, dropout=d))
                        model.add(layers.Dense(1, activation="sigmoid"))
                        model.compile(optimizer=o, loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy(),metrics.AUC(),metrics.Precision(),metrics.Recall(),metrics.F1Score(), metrics.TrueNegatives(), metrics.TruePositives(), metrics.FalseNegatives(), metrics.FalsePositives()])

                        start = time.perf_counter()
                        model.fit(X_train, y_train, epochs=25, callbacks=callbacks)
                        end = time.perf_counter()
                        train_time = end - start

                        start = time.perf_counter()
                        metric_results = model.evaluate(X_test, y_test)
                        end = time.perf_counter()
                        test_time = end - start

                        if a != 'relu' and np.isnan(metric_results[0]):
                            print(metric_results)

                        hp.hparams(hparams)
                        tf.summary.scalar(METRIC_BIN_ACC,metric_results[0], step=1)
                        tf.summary.scalar(METRIC_AUC,metric_results[1], step=1)
                        tf.summary.scalar(METRIC_PRECISION,metric_results[2], step=1)
                        tf.summary.scalar(METRIC_RECALL,metric_results[3], step=1)
                        tf.summary.scalar(METRIC_F1,metric_results[4], step=1)
                        tf.summary.scalar(METRIC_TN,metric_results[5][0], step=1)
                        tf.summary.scalar(METRIC_TP,metric_results[6], step=1)
                        tf.summary.scalar(METRIC_FN,metric_results[7], step=1)
                        tf.summary.scalar(METRIC_FP,metric_results[8], step=1)
                        tf.summary.scalar(METRIC_TRAIN_TIME, train_time, step=1)
                        tf.summary.scalar(METRIC_TEST_TIME, test_time, step=1)

                        keras.backend.clear_session()


Epoch 1/25
199/199 [==============================] - 120s 596ms/step - loss: 0.3288 - binary_accuracy: 0.8753 - auc: 0.4967 - precision: 0.0795 - recall: 0.0563 - f1_score: 0.1449 - true_negatives: 5540.0000 - true_positives: 28.0000 - false_negatives: 469.0000 - false_positives: 324.0000
Epoch 2/25
199/199 [==============================] - 126s 635ms/step - loss: 0.2745 - binary_accuracy: 0.9219 - auc: 0.4840 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.1449 - true_negatives: 5864.0000 - true_positives: 0.0000e+00 - false_negatives: 497.0000 - false_positives: 0.0000e+00
Epoch 3/25
199/199 [==============================] - 122s 613ms/step - loss: 0.2748 - binary_accuracy: 0.9219 - auc: 0.4937 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.1449 - true_negatives: 5864.0000 - true_positives: 0.0000e+00 - false_negatives: 497.0000 - false_positives: 0.0000e+00
Epoch 4/25
199/199 [==============================] - 172s 863ms/step - loss: 0.2751 - binary_accurac